In [3]:
%%capture
%%bash
. /opt/conda/bin/activat
conda install -c bioconda freebayes=1.3.1

In [7]:
%%bash
#freebayes -G 10 -F 0.3 -f p_ctg_cns_H1.fa reads_H1.bam > var_H1.vcf &
#freebayes -G 10 -F 0.3 -f p_ctg_cns_H2.fa reads_H2.bam > var_H2.vcf &

Process is interrupted.


In [ ]:
%%bash 
# this takes about one hours to run
freebayes-parallel <(fasta_generate_regions.py p_ctg_cns_H1.fa.fai 500000) 8 \
   -G 10 -F 0.3  -f p_ctg_cns_H1.fa reads_H1.bam > var_H1_2.vcf &

freebayes-parallel <(fasta_generate_regions.py p_ctg_cns_H2.fa.fai 500000) 8 \
   -G 10 -F 0.3  -f p_ctg_cns_H2.fa reads_H2.bam > var_H2_2.vcf &


In [3]:
%%bash
bgzip var_H1_2.vcf
bgzip var_H2_2.vcf


[bgzip] No such file or directory: var_H1_2.vcf


In [6]:
%%bash
tabix -p vcf var_H1_2.vcf.gz
tabix -p vcf var_H2_2.vcf.gz

In [14]:
import gzip

f = gzip.open("var_H1_2.vcf.gz")
v_pos = []
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    if len(row[3]) != len(row[4]):
        continue
    position = int(row[1])
    v_pos.append(position)

f = gzip.open("reads_H1.var.vcf.gz")
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    position = int(row[1])
    k, v = row[-2:]
    k = k.split(b':')
    v = v.split(b':')
    d = dict(zip(k,v))
    AD = [int(_) for _ in d[b'AD'].split(b',')]
    if min(AD)/sum(AD) < 0.25:
        continue
    pos = int(row[1])
    v_pos.append(position)

v_pos.sort()
SNP_cluster=[]
cluster_d = 10000
for position in v_pos:
    if len(SNP_cluster) == 0 or position - SNP_cluster[-1][-1] > cluster_d:
        SNP_cluster.append([position])
    else:
        SNP_cluster[-1].append(position)
for c in SNP_cluster:
    #print("H1", len(c), '000000F:{}-{}'.format(min(c)-50, max(c)+50), max(c)-min(c)+100)
    print('000000F\t{}\t{}'.format(min(c)-50, max(c)+50))
for c in SNP_cluster:
    print((min(c)-50, max(c)+50))

000000F	2241502	2241602
000000F	2389551	2389651
000000F	3475198	3475399
000000F	3486442	3508043
000000F	3956145	3969225
(2241502, 2241602)
(2389551, 2389651)
(3475198, 3475399)
(3486442, 3508043)
(3956145, 3969225)


In [15]:
f = gzip.open("var_H2_2.vcf.gz")
v_pos = []
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    if len(row[3]) != len(row[4]):
        continue
    position = int(row[1])
    v_pos.append(position)

f = gzip.open("reads_H2.var.vcf.gz")
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    position = int(row[1])
    k, v = row[-2:]
    k = k.split(b':')
    v = v.split(b':')
    d = dict(zip(k,v))
    AD = [int(_) for _ in d[b'AD'].split(b',')]
    if min(AD)/sum(AD) < 0.25:
        continue
    pos = int(row[1])
    v_pos.append(position)

v_pos.sort()
SNP_cluster=[]
cluster_d = 10000
for position in v_pos:
    if len(SNP_cluster) == 0 or position - SNP_cluster[-1][-1] > cluster_d:
        SNP_cluster.append([position])
    else:
        SNP_cluster[-1].append(position)
for c in SNP_cluster:
    #print("H2", len(c), '000000F:{}-{}'.format(min(c)-50, max(c)+50), max(c)-min(c)+100)
    print('000000F\t{}\t{}'.format(min(c)-50, max(c)+50))
for c in SNP_cluster:
    print((min(c)-50, max(c)+50))

000000F	2085306	2085406
000000F	2244234	2244334
000000F	2392104	2392204
000000F	3473257	3473557
000000F	3486094	3506188
000000F	3951022	3970406
(2085306, 2085406)
(2244234, 2244334)
(2392104, 2392204)
(3473257, 3473557)
(3486094, 3506188)
(3951022, 3970406)
